# Import modules

In [1]:
import pandas as pd

# our modules
import sys
sys.path.insert(0, '..')

# user defined
from clean import clean_data
from imbalance import balance_data
from evaluate import evaluate_model

# preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from imblearn.over_sampling import RandomOverSampler

# TensorFlow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier

# pipeline
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, roc_auc_score, r2_score, recall_score, precision_score, f1_score, matthews_corrcoef, confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt

2023-06-30 12:20:53.444130: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-30 12:20:53.468603: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-30 12:20:53.588213: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-30 12:20:53.589748: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 12:20:54.191002: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
pd.set_option("display.precision", 6)
pd.set_option("display.float_format", lambda x: "%.6f" % x)

In [3]:
RANDOM_STATE = 123

# Load data

In [4]:
data = pd.read_csv('/home/hristo/Documents/WBS_Data_Science/WBS_Predictive_Maintenance/data/ai4i2020.csv', index_col='UDI')

In [5]:
data

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
UDI,,,,,,,,,,,,,
1,M14860,M,298.100000,308.600000,1551,42.800000,0,0,0,0,0,0,0
2,L47181,L,298.200000,308.700000,1408,46.300000,3,0,0,0,0,0,0
3,L47182,L,298.100000,308.500000,1498,49.400000,5,0,0,0,0,0,0
4,L47183,L,298.200000,308.600000,1433,39.500000,7,0,0,0,0,0,0
5,L47184,L,298.200000,308.700000,1408,40.000000,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,M24855,M,298.800000,308.400000,1604,29.500000,14,0,0,0,0,0,0
9997,H39410,H,298.900000,308.400000,1632,31.800000,17,0,0,0,0,0,0
9998,M24857,M,299.000000,308.600000,1645,33.400000,22,0,0,0,0,0,0


# Clean data

In [6]:
data_cl = clean_data(data, drop_type=True)
data_cl

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure
UDI,,,,,,
1,298.100000,308.600000,1551,42.800000,0,0
2,298.200000,308.700000,1408,46.300000,3,0
3,298.100000,308.500000,1498,49.400000,5,0
4,298.200000,308.600000,1433,39.500000,7,0
5,298.200000,308.700000,1408,40.000000,9,0
...,...,...,...,...,...,...
9996,298.800000,308.400000,1604,29.500000,14,0
9997,298.900000,308.400000,1632,31.800000,17,0
9998,299.000000,308.600000,1645,33.400000,22,0


# Prepare data

In [7]:
X = data_cl.copy()
y = X.pop('Machine failure')

In [8]:
X

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
UDI,,,,,
1,298.100000,308.600000,1551,42.800000,0
2,298.200000,308.700000,1408,46.300000,3
3,298.100000,308.500000,1498,49.400000,5
4,298.200000,308.600000,1433,39.500000,7
5,298.200000,308.700000,1408,40.000000,9
...,...,...,...,...,...
9996,298.800000,308.400000,1604,29.500000,14
9997,298.900000,308.400000,1632,31.800000,17
9998,299.000000,308.600000,1645,33.400000,22


## Scale data

In [9]:
def scale_data(data):
    data = data.copy()
    return StandardScaler().fit_transform(data)

In [10]:
X_sc = scale_data(X)

In [11]:
X_sc = pd.DataFrame(X_sc, index=X.index, columns=X.columns)

In [12]:
X_sc

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
UDI,,,,,
1,-0.952389,-0.947360,0.068185,0.282200,-1.695984
2,-0.902393,-0.879959,-0.729472,0.633308,-1.648852
3,-0.952389,-1.014761,-0.227450,0.944290,-1.617430
4,-0.902393,-0.947360,-0.590021,-0.048845,-1.586009
5,-0.902393,-0.879959,-0.729472,0.001313,-1.554588
...,...,...,...,...,...
9996,-0.602417,-1.082162,0.363820,-1.052012,-1.476034
9997,-0.552421,-1.082162,0.520005,-0.821283,-1.428902
9998,-0.502425,-0.947360,0.592519,-0.660777,-1.350349


# Split train and test

In [13]:
balance = False
if balance:
    data_bal, method = balance_data(X_sc, y, RANDOM_STATE, 'ROS')
    X_bl, y_bl = data_bal
    # y_bl.value_counts().plot(kind='bar')

    X_train, X_test, y_train, y_test = train_test_split(X_bl, y_bl, train_size=0.8, random_state=RANDOM_STATE)
else:
    X_train, X_test, y_train, y_test = train_test_split(X_sc, y, train_size=0.8, random_state=RANDOM_STATE)

# Baseline model

In [14]:
early_stopping = EarlyStopping(
    min_delta=0.001,
    patience=40,
    monitor='loss',
    restore_best_weights=True,
)

In [15]:
def baseline_model(input_shape, n_nodes=512, n_hidden=2):
    ''' create model '''

    model = Sequential()
    model.add(Dense(n_nodes, activation='relu', input_shape=[input_shape]))
    for i in range(n_hidden):
        model.add(Dropout(0.3))
        model.add(BatchNormalization())
        model.add(Dense(n_nodes, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [33]:
classifier = KerasClassifier(
    model=baseline_model(input_shape=X_train.shape[1], n_hidden=3),
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
    epochs=100,
    callbacks=early_stopping,
    random_state=RANDOM_STATE,
    verbose=False
)

In [34]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', classifier)
])

In [35]:
pipeline.fit(X_train, y_train)

2023-06-30 13:51:36.875833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [8000,1]
	 [[{{node Placeholder/_11}}]]
2023-06-30 13:51:36.875986: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [8000,1]
	 [[{{node Placeholder/_11}}]]


Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 KerasClassifier(callbacks=<keras.callbacks.EarlyStopping object at 0x7f042c93c990>, epochs=100, loss='binary_crossentropy', metrics=['binary_accuracy'], model=<keras.engine.sequential.Sequential object at 0x7f032c1e0b90>, optimizer='adam', random_state=123, verbose=False))])

In [36]:
y_pred = pipeline.predict(X_test)

In [37]:
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy - {100*accuracy:.2f} %')

accuracy - 98.10 %


# Grid

In [38]:
optimizers = ['rmsprop', 'adam']
# init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
# batches = [5, 10, 20]
param_grid = dict(
    classifier__optimizer=optimizers,
    classifier__epochs=epochs,
    # classifier__init=init
)

In [39]:
def run_model(pipe, params, X_train, y_train, X_test):
    ''' compute deep learning model(s)'''

    grid = GridSearchCV(
        estimator=pipe,
        param_grid=params,
        scoring='accuracy',
        verbose=1
    )

    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    best_params = grid.best_params_

    return best_model, best_params

In [40]:
best_model, best_params = run_model(pipeline, param_grid, X_train, y_train, X_test)

2023-06-30 13:54:38.737732: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 13:54:38.745033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 13:54:38.752763: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 13:54

INFO:tensorflow:Assets written to: ram://3c0e983c6cf64ee0b6e507f5bb7e427a/assets


INFO:tensorflow:Assets written to: ram://3c0e983c6cf64ee0b6e507f5bb7e427a/assets
2023-06-30 13:54:39.795440: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://136994ad764c4b02aa87da151c4f1a6b: INVALID_ARGUMENT: ram://136994ad764c4b02aa87da151c4f1a6b is a directory.
2023-06-30 13:54:40.141927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 13:54:40.149691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 13:54:40.157032: I tensorflow/core/common_runt

INFO:tensorflow:Assets written to: ram://a01289d97dd84268abac8d6b059773ad/assets


INFO:tensorflow:Assets written to: ram://a01289d97dd84268abac8d6b059773ad/assets
2023-06-30 13:54:41.610417: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://2489c864c4a84463beda9e19324267ea: INVALID_ARGUMENT: ram://2489c864c4a84463beda9e19324267ea is a directory.


...

INFO:tensorflow:Assets written to: ram://c4ca356ebce643fea462762b1100d324/assets
2023-06-30 14:36:45.778017: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://e06d488d2c3948e882eab63f10afed67: INVALID_ARGUMENT: ram://e06d488d2c3948e882eab63f10afed67 is a directory.
2023-06-30 14:36:46.195957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 14:36:46.205301: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 14:36:46.213624: I tensorflow/core/common_runt

INFO:tensorflow:Assets written to: ram://2a805aba4b7244cba31361226a0e7651/assets


INFO:tensorflow:Assets written to: ram://2a805aba4b7244cba31361226a0e7651/assets
2023-06-30 14:36:47.985498: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://61a6064b14154bfea3137d1da195e19a: INVALID_ARGUMENT: ram://61a6064b14154bfea3137d1da195e19a is a directory.
2023-06-30 14:36:48.406991: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 14:36:48.417554: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,512]
	 [[{{node inputs}}]]
2023-06-30 14:36:48.427351: I tensorflow/core/common_runt

INFO:tensorflow:Assets written to: ram://91279d4b2d6345e2a23a9f517b69f61a/assets


INFO:tensorflow:Assets written to: ram://91279d4b2d6345e2a23a9f517b69f61a/assets
2023-06-30 14:36:49.681568: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ram://8a165bb0dfaf4824a288e1245113a4b6: INVALID_ARGUMENT: ram://8a165bb0dfaf4824a288e1245113a4b6 is a directory.
2023-06-30 14:36:49.780711: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [8000,1]
	 [[{{node Placeholder/_11}}]]
2023-06-30 14:36:49.780934: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [8000,1]
	 [[{{node Placeholder/_11}}]]


In [41]:
best_params

{'classifier__epochs': 100, 'classifier__optimizer': 'rmsprop'}

In [42]:
predictions = best_model.predict(X_test)
predictions = pd.DataFrame(predictions.round(), index=y_test.index)

In [43]:
accuracy = accuracy_score(y_test, predictions)
print(100*accuracy)

97.95


# for model history

In [ ]:
def create_model(n_nodes, n_hidden, input_shape, optimizer, n_epochs, loss='binary_crossentropy', metrics=['binary_accuracy']):

    model = Sequential()
    model.add(Dense(n_nodes, activation='relu', input_shape=[input_shape]))
    for i in range(n_hidden):
        model.add(Dense(n_nodes, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        batch_size=int(n_nodes/2),
        epochs=n_epochs,
        callbacks=[early_stopping],
        verbose=0
    )

    predictions = model.predict(X_test)
    predictions = pd.DataFrame(predictions.round(), index=y_test.index)

    return history, predictions

In [ ]:
history, predictions = create_model(512, 2, X_train.shape[1], 'adam', 150)

In [ ]:
history_df = pd.DataFrame(history.history)

In [ ]:
# history_df.loc[:, ['loss', 'val_loss']].plot()
# plt.ylim([0, 1])
history_df.loc[:, ['loss', 'val_loss']].plot()
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()
print(f"Minimum validation loss: {history_df['val_loss'].min():.6%}")
print(f"Maximum validation accuracy: {history_df['val_binary_accuracy'].max():.6%}")
print(f"Maximum binary accuracy: {history_df['binary_accuracy'].max():.6%}")

In [ ]:
y_pred = predictions

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy - {accuracy:.2f} %')